In [1]:
import aup
import pandas as pd
from keplergl import KeplerGl
import geopandas as gpd

In [24]:
df = pd.read_json("../scripts/Metropolis_CVE.json")
cities = [c for c in df.columns.unique()]
# Creates empty GeoDataFrame to store specified municipality polygons
mun_gdf = gpd.GeoDataFrame()
#ageb_gdf = gpd.GeoDataFrame()
hex_bins = gpd.GeoDataFrame()
year = 2020
mpos_folder = f'mpos_{year}'

c = cities[-1]
for i in range(len(df.loc["mpos", c])):
    # Extracts specific municipality code
    m = df.loc["mpos", c][i]
    # Downloads municipality polygon according to code
    query = f"SELECT * FROM marco.{mpos_folder} WHERE \"CVEGEO\" LIKE \'{m}\'"
    mun_gdf = mun_gdf.append(aup.gdf_from_query(query, geometry_col='geometry'))
    #Creates query to download hex bins
    query = f"SELECT * FROM processed.hex_bins_index_{year} WHERE \"CVEGEO\" LIKE \'{m}%%\'"
    hex_bins = hex_bins.append(aup.gdf_from_query(query, geometry_col='geometry'))

In [25]:
map_city = KeplerGl(height=800, data={'data_farmacias':hex_bins}, config=config)
#map_city.add_data(hex_bins, c)
map_city

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


/Users/natera/miniconda3/envs/ox/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'jsx1yd', 'type': '…

In [30]:
map_city.save_to_html(file_name=f'../output/html/{c}_farmacias.html')

Map saved to ../output/html/Zacatecas_farmacias.html!


/Users/natera/miniconda3/envs/ox/lib/python3.8/site-packages/geopandas/geodataframe.py:852: UserWarning: Geometry column does not contain geometry.
  warnings.warn("Geometry column does not contain geometry.")


In [10]:
with open('hex_config.py', 'w') as f:
   f.write('config = {}'.format(map_city.config))


In [20]:
%run hex_config.py

In [21]:
config

{'version': 'v1',
 'config': {'visState': {'filters': [],
   'layers': [{'id': 'jsx1yd',
     'type': 'geojson',
     'config': {'dataId': 'data_farmacias',
      'label': 'data_farmacias',
      'color': [231, 159, 213],
      'columns': {'geojson': 'geometry'},
      'isVisible': True,
      'visConfig': {'opacity': 0.35,
       'strokeOpacity': 0.18,
       'thickness': 0.2,
       'strokeColor': [255, 254, 230],
       'colorRange': {'name': 'Custom Palette',
        'type': 'custom',
        'category': 'Custom',
        'colors': ['#FAE300', '#FD7900', '#CF1750', '#7A0DA6', '#2C51BE']},
       'strokeColorRange': {'name': 'Global Warming',
        'type': 'sequential',
        'category': 'Uber',
        'colors': ['#5A1846',
         '#900C3F',
         '#C70039',
         '#E3611C',
         '#F1920E',
         '#FFC300']},
       'radius': 10,
       'sizeRange': [0, 10],
       'radiusRange': [0, 50],
       'heightRange': [0, 500],
       'elevationScale': 5,
       'stroked

In [27]:
config['config']['mapState']

{'bearing': 0,
 'dragRotate': False,
 'latitude': 20.663763061180177,
 'longitude': -103.37697537836645,
 'pitch': 0,
 'zoom': 9.193984242046747,
 'isSplit': False}